<a href="https://colab.research.google.com/github/LorenzoBellomo/InformationRetrieval/blob/main/notebooks/5_GraphDBs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neo4j

In [2]:
!pip install yfiles_jupyter_graphs

In [19]:
from neo4j import GraphDatabase

[https://sandbox.neo4j.com/](https://sandbox.neo4j.com/)

In [23]:
driver = GraphDatabase.driver("bolt://34.234.236.176:7687", auth=("neo4j", "rush-history-enlistments"))
session = driver.session(database="neo4j")
driver.execute_query("MATCH (n) DETACH DELETE n  ") # purging eventual graphs already loaded

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x781f18eb3ed0>, keys=[])

In [24]:
driver.execute_query(
    "CREATE (john:Person {name: \"John\"})   \
    CREATE (joe:Person {name: \"Joe\"})      \
    CREATE (steve:Person {name: \"Steve\"})  \
    CREATE (sara:Person {name: \"Sara\"})    \
    CREATE (maria:Person {name: \"Maria\"})  \
    CREATE (john)-[:FRIEND]->(joe)-[:FRIEND]->(steve)    \
    CREATE (john)-[:FRIEND]->(sara)-[:FRIEND]->(maria)   \
")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x781f18d207d0>, keys=[])

In [26]:
results = driver.execute_query("MATCH (n) return n")

In [34]:
#printing all the properties "name"
for record in results[0]:
    elem = record["n"]
    print("{} node with name {}".format(list(elem.labels)[0], elem["name"]))

Person node with name Steve
Person node with name Sara
Person node with name Maria
Person node with name John
Person node with name Joe


In [45]:
from yfiles_jupyter_graphs import GraphWidget
from google.colab import output
output.enable_custom_widget_manager()
def draw_graph():
    g = session.run("MATCH (n) -[e]-> (m) return n, m, e").graph()
    gw = GraphWidget(graph=g)
    return gw

draw_graph()

In [47]:
results = driver.execute_query("CREATE (j:Person {name: \"Sara\"})-[rel:FRIEND]->(m:Person {name: \"Joe\"})  ")
draw_graph()

In [73]:
results = (driver.execute_query("MATCH (n {name: \"Sara\"}) -[]-> (m {name: \"Joe\"}) RETURN n, m"))
res = results[0][0]
print(res)
print(res["n"].element_id)
print(res["m"].element_id)
driver.execute_query("MATCH (n) where elementId(n)=$id DETACH DELETE n  ", id=res["n"].element_id)
driver.execute_query("MATCH (n) where elementId(n)=$id DETACH DELETE n  ", id=res["m"].element_id)
draw_graph()

<Record n=<Node element_id='4:2271f349-2594-48d6-be21-c7ad09a910f7:3' labels=frozenset({'Person'}) properties={'name': 'Sara'}> m=<Node element_id='4:2271f349-2594-48d6-be21-c7ad09a910f7:4' labels=frozenset({'Person'}) properties={'name': 'Joe'}>>
4:2271f349-2594-48d6-be21-c7ad09a910f7:3
4:2271f349-2594-48d6-be21-c7ad09a910f7:4


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x781f18afca90>, keys=[])

In [75]:
driver.execute_query("MATCH (j:Person {name: \"Sara\"})  MATCH (m:Person {name: \"Joe\"})  MERGE (j)-[r:FRIEND]->(m)  RETURN j, r, m ")
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [77]:
driver.execute_query("CREATE (aus:Country {name: \"Australia\"})  CREATE (ger:Country {name: \"Germany\"}) ")
driver.execute_query("MATCH (j:Person {name: \"Sara\"})  MATCH (l:Country {name: \"Germany\"})  MERGE (j)-[r:LIVES_IN]->(l)  RETURN j, l ")
driver.execute_query("MATCH (j:Person {name: \"Maria\"})  MATCH (l:Country {name: \"Germany\"})  MERGE (j)-[r:LIVES_IN]->(l)  RETURN j, l ")
driver.execute_query("MATCH (j:Person {name: \"John\"})  MATCH (l:Country {name: \"Germany\"})  MERGE (j)-[r:LIVES_IN]->(l)  RETURN j, l ")
driver.execute_query("MATCH (j:Person {name: \"Joe\"})  MATCH (l:Country {name: \"Australia\"})  MERGE (j)-[r:LIVES_IN]->(l)  RETURN j, l ")
driver.execute_query("MATCH (j:Person {name: \"Steve\"})  MATCH (l:Country {name: \"Australia\"})  MERGE (j)-[r:LIVES_IN]->(l)  RETURN j, l ")
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [81]:
results = driver.execute_query("MATCH (p:Person) WHERE p.name =~ 'Jo.*' RETURN p.name  ")
for res in results[0]:
    print(res["p.name"])

John
Joe


In [83]:
results = driver.execute_query("MATCH (p:Person)-[r:FRIEND]->(m:Person)  WHERE exists((m)-[:FRIEND]->(p))  RETURN m, p ")
for res in results[0]:
    print(res["m"]["name"], " -> ", res["p"]["name"])

In [92]:
results = driver.execute_query("MATCH (p1:Person)-[:FRIEND]->(p2:Person) RETURN p1.name, collect(p2.name) as listOfFriends, count(*) as numberOfFriends")
for res in results[0]:
  print("{} has {} friends {}".format(res["p1.name"], res["numberOfFriends"], res["listOfFriends"]))

Person: Sara has 2 friends ['Maria', 'Joe']
Person: John has 2 friends ['Sara', 'Joe']
Person: Joe has 1 friends ['Steve']


**EX1:** Count the number of people who have at least one friend that lives in Australia

**EX2:** Print the names of people who have at least one friend that lives in Australia

In [94]:
q = "TODO - ADD QUERY HERE "
results = driver.execute_query(q)
for res in results[0]:
    print(res)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'TODO': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"TODO - ADD QUERY HERE"
 ^}

In [99]:
results = driver.execute_query("MATCH (p:Person {name: \"Maria\"})-[relatedTo]-(x)  return x.name, type(relatedTo), Labels(x)[0]")
for res in results[0]:
  print("Maria -[{}]-> {} ({})".format(res["type(relatedTo)"], res["x.name"], res["Labels(x)[0]"]))

Maria -[FRIEND]-> Sara (Person)
Maria -[LIVES_IN]-> Germany (Country)
Maria -[LIVES_IN]-> Germany (Country)


In [102]:
results = driver.execute_query("MATCH (p:Person {name: \"Maria\"})-[*1..2]-(x)  return DISTINCT x ")
for res in results[0]:
  print(res["x"])

<Node element_id='4:2271f349-2594-48d6-be21-c7ad09a910f7:1' labels=frozenset({'Person'}) properties={'name': 'Sara'}>
<Node element_id='4:2271f349-2594-48d6-be21-c7ad09a910f7:6' labels=frozenset({'Country'}) properties={'name': 'Germany'}>
<Node element_id='4:2271f349-2594-48d6-be21-c7ad09a910f7:8' labels=frozenset({'Country'}) properties={'name': 'Germany'}>
<Node element_id='4:2271f349-2594-48d6-be21-c7ad09a910f7:11' labels=frozenset({'Person'}) properties={'name': 'Joe'}>
<Node element_id='4:2271f349-2594-48d6-be21-c7ad09a910f7:10' labels=frozenset({'Person'}) properties={'name': 'John'}>


In [108]:
results = driver.execute_query("MATCH (p:Person {name: \"Maria\"})-[*1..2]->(x)  return DISTINCT x ")
for res in results[0]:
  print(res["x"])

<Node element_id='4:2271f349-2594-48d6-be21-c7ad09a910f7:6' labels=frozenset({'Country'}) properties={'name': 'Germany'}>
<Node element_id='4:2271f349-2594-48d6-be21-c7ad09a910f7:8' labels=frozenset({'Country'}) properties={'name': 'Germany'}>


In [107]:
g = session.run("CALL db.schema.visualization()").graph()
GraphWidget(graph=g)

GraphWidget(layout=Layout(height='500px', width='100%'))

In [115]:
results = driver.execute_query("CALL gds.graph.project(\'provass\', [\'Person\', \'Country\'], [\'FRIEND\', \'LIVES_IN\']) YIELD graphName AS graph, nodeProjection, nodeCount AS nodes, relationshipCount AS rels  ")
print(results[0])

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: A graph with name 'provass' already exists.}

In [116]:
results = driver.execute_query("CALL gds.pageRank.stream(\'projection_name\') YIELD nodeId, score WITH gds.util.asNode(nodeId) AS n,score AS pageRank RETURN n.name AS name, Labels(n)[0] AS type, pageRank ORDER BY pageRank DESC ")
for res in results[0]:
  print(res["name"], " - ", res["type"], " - ", res["pageRank"])

Steve  -  Person  -  0.337444921875
Germany  -  Country  -  0.3006990234375
Germany  -  Country  -  0.3006990234375
Joe  -  Person  -  0.22052343750000003
Maria  -  Person  -  0.18864843750000002
Sara  -  Person  -  0.181875
Australia  -  Country  -  0.15000000000000002
Australia  -  Country  -  0.15000000000000002
John  -  Person  -  0.15000000000000002


In [118]:
results = driver.execute_query("CALL gds.pageRank.write('projection_name',     {writeProperty: 'pageRank'} ) YIELD nodePropertiesWritten, ranIterations ")
results = driver.execute_query("MATCH (n) RETURN n.name as name, n.pageRank as pr")
for res in results[0]:
  print(res["name"], " - ", res["pr"])

Steve  -  0.337444921875
Sara  -  0.181875
Maria  -  0.18864843750000002
Australia  -  0.15000000000000002
Germany  -  0.3006990234375
Australia  -  0.15000000000000002
Germany  -  0.3006990234375
John  -  0.15000000000000002
Joe  -  0.22052343750000003


In [120]:
results = driver.execute_query("MATCH (source:Person {name: \'Maria\'})MATCH (target:Person {name: \'Joe\'})CALL gds.shortestPath.dijkstra.stream(\'projection_name\', {sourceNode: source,targetNode: target})YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, pathRETURNindex, gds.util.asNode(sourceNode).name AS SourceName,gds.util.asNode(targetNode).name AS targetNodeName,totalCost, [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,costs, nodes(path) as pathORDER BY index")
for res in results[0]:
    print(res)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '.': expected 'FOREACH', ',', 'AS', 'ORDER BY', 'CALL', 'CREATE', 'LOAD CSV', 'DELETE', 'DETACH', 'FINISH', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REMOVE', 'RETURN', 'SET', 'SKIP', 'UNION', 'UNWIND', 'USE', 'WHERE', 'WITH' or <EOF> (line 1, column 254 (offset: 253))
"MATCH (source:Person {name: 'Maria'})MATCH (target:Person {name: 'Joe'})CALL gds.shortestPath.dijkstra.stream('projection_name', {sourceNode: source,targetNode: target})YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, pathRETURNindex, gds.util.asNode(sourceNode).name AS SourceName,gds.util.asNode(targetNode).name AS targetNodeName,totalCost, [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,costs, nodes(path) as pathORDER BY index"
                                                                                                                                                 